Numerical Analysis Project - Erya Jain

In [7]:
import numpy as np import matplotlib.pyplot as plt from math import log, sqrt, exp, erf def norm_cdf(x: float) -> float: return 0.5 * (1.0 + erf(x / sqrt(2.0))) def bs_european_call_price(S0: float, K: float, r: float, sigma: float, T: float) -> float: """Black–Scholes European call price.""" if T <= 0: return max(S0 - K, 0.0) d1 = (log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T)) d2 = d1 - sigma * sqrt(T) return S0 * norm_cdf(d1) - K * exp(-r * T) * norm_cdf(d2) def geometric_asian_call_price_discrete(S0: float, K: float, r: float, sigma: float, T: float, m: int) -> float: if T <= 0: return max(S0 - K, 0.0) t_bar = T * (m + 1) / (2.0 * m) sum_min = T * (m + 1) * (2 * m + 1) / 6.0 var_G = (sigma**2) * (sum_min / (m**2)) mu_G = log(S0) + (r - 0.5 * sigma**2) * t_bar sig_G = sqrt(var_G) if sig_G == 0: G_det = exp(mu_G) return exp(-r * T) * max(G_det - K, 0.0) d1 = (mu_G - log(K) + var_G) / sig_G d2 = d1 - sig_G price = exp(-r * T) * (exp(mu_G + 0.5 * var_G) * norm_cdf(d1) - K * norm_cdf(d2)) return price def summarize_estimator(payoffs: np.ndarray, name: str = "") -> dict: N = len(payoffs) price = float(np.mean(payoffs)) var_payoff = float(np.var(payoffs, ddof=1)) se = float(np.sqrt(var_payoff / N)) return {"method": name, "price": price, "var_payoff": var_payoff, "se": se} def simulate_gbm_paths_exact(S0: float, r: float, sigma: float, T: float, m_steps: int, Z: np.ndarray) -> np.ndarray: dt = T / m_steps drift = (r - 0.5 * sigma**2) * dt diffusion = sigma * np.sqrt(dt) * Z logS = np.cumsum(drift + diffusion, axis=1) S = np.empty((Z.shape[0], m_steps + 1)) S[:, 0] = S0 S[:, 1:] = S0 * np.exp(logS) return S def run_experiment(S0, K, r, sigma, T, N, m_steps, seed=12345): rng = np.random.default_rng(seed) discount = np.exp(-r * T) tau_euro = bs_european_call_price(S0, K, r, sigma, T) tau_geo = geometric_asian_call_price_discrete(S0, K, r, sigma, T, m_steps) results = {} Z_euro = rng.standard_normal(N) ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z_euro) X_euro = discount * np.maximum(ST - K, 0.0) results["Euro Std MC"] = summarize_estimator(X_euro, "Euro Std MC") ST_anti = S0 * np.exp((r - 0.5 * sigma**2) * T - sigma * np.sqrt(T) * Z_euro) X_euro_av = 0.5 * (X_euro + discount * np.maximum(ST_anti - K, 0.0)) results["Euro Antithetic"] = summarize_estimator(X_euro_av, "Euro Antithetic") t_euro = discount * ST cov_e = np.cov(X_euro, t_euro, ddof=1)[0, 1] var_X = np.var(X_euro, ddof=1) var_t = np.var(t_euro, ddof=1) c_star = -cov_e / var_t if var_t > 0 else 0.0 X_euro_cv = X_euro + c_star * (t_euro - S0) cv_res = summarize_estimator(X_euro_cv, "Euro CV") cv_res["c_star"] = float(c_star) cv_res["rho"] = float(cov_e / np.sqrt(var_X * var_t)) if (var_X > 0 and var_t > 0) else np.nan results["Euro CV"] = cv_res Z_asian = rng.standard_normal((N, m_steps)) S_path = simulate_gbm_paths_exact(S0, r, sigma, T, m_steps, Z_asian) arith_avg = np.mean(S_path[:, 1:], axis=1) X_asian = discount * np.maximum(arith_avg - K, 0.0) results["Asian Std MC"] = summarize_estimator(X_asian, "Asian Std MC") S_path_anti = simulate_gbm_paths_exact(S0, r, sigma, T, m_steps, -Z_asian) arith_avg_anti = np.mean(S_path_anti[:, 1:], axis=1) X_asian_av = 0.5 * (X_asian + discount * np.maximum(arith_avg_anti - K, 0.0)) results["Asian Antithetic"] = summarize_estimator(X_asian_av, "Asian Antithetic") ST_asian = S_path[:, -1] t_euro_asian = discount * np.maximum(ST_asian - K, 0.0) cov1 = np.cov(X_asian, t_euro_asian, ddof=1)[0, 1] var1 = np.var(t_euro_asian, ddof=1) c1 = -cov1 / var1 if var1 > 0 else 0.0 X_cv1 = X_asian + c1 * (t_euro_asian - tau_euro) res1 = summarize_estimator(X_cv1, "Asian CV (European)") res1["c_star"] = float(c1) res1["rho"] = float(np.corrcoef(X_asian, t_euro_asian)[0, 1]) results["Asian CV (European)"] = res1 geo_avg = np.exp(np.mean(np.log(S_path[:, 1:]), axis=1)) t_geo = discount * np.maximum(geo_avg - K, 0.0) cov2 = np.cov(X_asian, t_geo, ddof=1)[0, 1] var2 = np.var(t_geo, ddof=1) c2 = -cov2 / var2 if var2 > 0 else 0.0 X_cv2 = X_asian + c2 * (t_geo - tau_geo) res2 = summarize_estimator(X_cv2, "Asian CV (Geometric)") res2["c_star"] = float(c2) res2["rho"] = float(np.corrcoef(X_asian, t_geo)[0, 1]) results["Asian CV (Geometric)"] = res2 return results, tau_euro, tau_geo def plot_se_vs_N(N_list, se_series, title): plt.figure(figsize=(8, 6)) for method, se_vals in se_series.items(): plt.plot(N_list, se_vals, marker="o", label=method) plt.xscale("log") plt.yscale("log") plt.xlabel("Number of simulations") plt.ylabel("Standard error") plt.title(title) plt.legend() plt.grid(True, which="both", ls="--", linewidth=0.5) plt.show() if __name__ == "__main__": S0, K, r, sigma, T = 100.0, 100.0, 0.05, 0.2, 1.0 m_steps = 252 N_list = [5_000, 10_000, 25_000, 50_000, 100_000] se_euro = {"Std MC": [], "Antithetic": [], "CV": []} se_asian = {"Std MC": [], "Antithetic": [], "CV (European)": [], "CV (Geometric)": []} final_results = None final_tau_euro = None final_tau_geo = None for N in N_list: results, tau_euro, tau_geo = run_experiment(S0, K, r, sigma, T, N, m_steps, seed=12345 + N) se_euro["Std MC"].append(results["Euro Std MC"]["se"]) se_euro["Antithetic"].append(results["Euro Antithetic"]["se"]) se_euro["CV"].append(results["Euro CV"]["se"]) se_asian["Std MC"].append(results["Asian Std MC"]["se"]) se_asian["Antithetic"].append(results["Asian Antithetic"]["se"]) se_asian["CV (European)"].append(results["Asian CV (European)"]["se"]) se_asian["CV (Geometric)"].append(results["Asian CV (Geometric)"]["se"]) if N == N_list[-1]: final_results = results final_tau_euro = tau_euro final_tau_geo = tau_geo N_final = N_list[-1] print(f"\nBlack–Scholes European Call Price: {final_tau_euro:.6f}") print(f"Geometric Asian Call: {final_tau_geo:.6f}") print(f"Results at N = {N_final:,}\n") print(f"{'Method':<52} {'Price':>12} {'Var(payoff)':>16} {'SE':>14}") print("-" * 94) for key in ["Euro Std MC", "Euro Antithetic", "Euro CV"]: row = final_results[key] name = row["method"] if "CV" in key: name += f" (c*≈{row['c_star']:.3f}, ρ≈{row['rho']:.6f})" print(f"{name:<52} {row['price']:12.6f} {row['var_payoff']:16.4f} {row['se']:14.6f}") print("") for key in ["Asian Std MC", "Asian Antithetic", "Asian CV (European)", "Asian CV (Geometric)"]: row = final_results[key] name = row["method"] if "CV" in key: name += f" (c*≈{row['c_star']:.3f}, ρ≈{row['rho']:.6f})" print(f"{name:<52} {row['price']:12.6f} {row['var_payoff']:16.4f} {row['se']:14.6f}") plot_se_vs_N(N_list, se_euro, "European Call: Standard Error vs N") plot_se_vs_N(N_list, se_asian, "Asian Call: Standard Error vs N")


Black–Scholes European Call Price: 10.450584
Geometric Asian Call:       5.565509
Results at N = 100,000

Method                                                      Price      Var(payoff)             SE
----------------------------------------------------------------------------------------------
Euro Std MC                                             10.498568         217.2102       0.046606
Euro Antithetic                                         10.423607          53.9187       0.023220
Euro CV (c*≈-0.676, ρ≈0.924998)                         10.424134          31.3604       0.017709

Asian Std MC                                             5.788829          63.7022       0.025239


TypeError: plot_se_vs_N() takes 3 positional arguments but 4 were given